In [15]:
from sklearn.model_selection import cross_val_score, cross_validate, StratifiedKFold, ShuffleSplit
from sklearn.metrics import accuracy_score, balanced_accuracy_score, recall_score, precision_score, confusion_matrix, make_scorer
from sklearn.svm import SVC
from pennylane import numpy as np
import pennylane as qml
import pandas as pd
from utils import specificity_score, negative_prediction_value_score, gmean_score, informedness_score

# Data loading

In [16]:
# Load the data
df = pd.read_csv('./data/HTRU_2.csv', header=None)
df.columns = ['IpMean', 'IpDev', 'IpKurt','IpSkew', 'DMMean', 'DMDev', 'DMKurt', 'DMSkew', 'Class']

# Split the data into features and target
X = df.iloc[:, :-1]
y = df.iloc[:, -1]

# Define the number of experiments
n_runs = 5

# Initialize lists to store the scores
scores = {
    'accuracy': [],
    'balanced_accuracy': [],
    'recall': [],
    'specificity': [],
    'precision': [],
    'npv': [],
    'gmean': [],
    'informedness': []
}


# Create a ShuffleSplit instance
ss = ShuffleSplit(n_splits=n_runs, train_size=200, test_size=400)

# Kernel definition

In [17]:
n_qubits = 8

dev = qml.device("lightning.qubit", wires=n_qubits) 


# kernel les angles
@qml.qnode(dev)
def kernel_angle_embedding(a,b):
    # phi a
    qml.AngleEmbedding(
        a, wires=range(n_qubits)
    )
    # phi b
    qml.AngleEmbedding(
        b, wires=range(n_qubits)
    )

    return qml.probs(wires=range(n_qubits))


def qkernel(A, B):
    # Ensure that A and B are NumPy arrays for compatibility with the quantum kernel
    A = np.array(A)
    B = np.array(B)
    
    # Compute the pairwise kernel values
    kernel_matrix = np.zeros((A.shape[0], B.shape[0]))
    for i, a in enumerate(A):
        for j, b in enumerate(B):
            kernel_matrix[i, j] = kernel_angle_embedding(a, b)[0]
    return kernel_matrix

# Training QSVM

In [18]:

# Initialize the model
model = SVC(kernel=qkernel, C=1.0) 

for train_index, test_index in ss.split(X):
    # Split the data into training and testing sets
    x_train, x_test = X.iloc[train_index], X.iloc[test_index]
    y_train, y_test = y.iloc[train_index], y.iloc[test_index]

    # Train the model and make predictions
    model.fit(x_train, y_train)
    y_pred = model.predict(x_test)

    # Store the scores
    scores['accuracy'].append(accuracy_score(y_test, y_pred))
    scores['balanced_accuracy'].append(balanced_accuracy_score(y_test, y_pred))
    scores['recall'].append(recall_score(y_test, y_pred))
    scores['specificity'].append(specificity_score(y_test, y_pred))
    scores['precision'].append(precision_score(y_test, y_pred))
    scores['npv'].append(negative_prediction_value_score(y_test, y_pred))
    scores['gmean'].append(gmean_score(y_test, y_pred))
    scores['informedness'].append(informedness_score(y_test, y_pred))

# Prediction

In [ ]:
# Calculate mean and standard deviation for each metric and print the results
for metric, values in scores.items():
    mean_value = np.mean(values)
    std_value = np.std(values)
    print(f"{metric.capitalize()}: {mean_value:.3f} ± {std_value:.3f}")

Accuracy: 0.925 ± 0.000
Balanced_accuracy: 0.500 ± 0.000
Recall: 0.000 ± 0.000
Specificity: 1.000 ± 0.000
Precision: 0.000 ± 0.000
Npv: 0.925 ± 0.000
Gmean: 0.000 ± 0.000
Informedness: 0.000 ± 0.000


# Prediction

In [ ]:
# Calculate mean and standard deviation for each metric and print the results
for metric, values in scores.items():
    mean_value = np.mean(values)
    std_value = np.std(values)
    print(f"{metric.capitalize()}: {mean_value:.3f} ± {std_value:.3f}")